In [ ]:
import os
import os
import numpy as np
import pickle
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import re
import time
from Util import Env, quantizer, shifter, get_parser
datadir = 'lotr/04-crypto-sc/data/'

In [ ]:
onefiles = os.listdir(datadir + 'eddsa-1-parsed')
onelist = []
for fname in onefiles:
    with open(datadir + 'eddsa-1-parsed/' + fname, 'r') as f:
        onelist.append([int(n) for n in f])
print(len(onelist))
onethreshold = np.percentile(np.array([len(l) for l in onelist]),0.3)

zerofiles = os.listdir(datadir + 'eddsa-0-parsed')
zerolist = []
for fname in zerofiles:
    with open(datadir + 'eddsa-0-parsed/' + fname, 'r') as f:
        zerolist.append([int(n) for n in f])
print(len(zerolist))
zerothreshold = np.percentile(np.array([len(l) for l in zerolist]),0.3)
print((onethreshold, zerothreshold))

In [ ]:
threshold = int(min(zerothreshold, onethreshold))
newzero = [z for z in zerolist if len(z) >= threshold]
newone = [o for o in onelist if len(o) >= threshold]
dsize = min((len(newzero),len(newone)))
x_arr = np.zeros([dsize*2, threshold],dtype=np.float32)
y_arr = np.arange(dsize*2)%2
for i in range(dsize):
    x_arr[2*i] = newzero[i][:threshold]
    x_arr[2*i+1] = newone[i][:threshold]
print(x_arr.shape)

In [ ]:
std = 16
mean = np.median(x_arr)
x_arr2 = (x_arr-mean)/std
print((mean,))

In [ ]:
train_x = x_arr2[:4000]
train_y = y_arr[:4000]
valid_x = x_arr2[4000:6000]
valid_y = y_arr[4000:6000]
test_x = x_arr2[6000:7000]
test_y = y_arr[6000:7000]
from sklearn import svm
clf = svm.SVC(kernel = 'rbf',gamma=0.001, C=0.5)

clf.fit(train_x, train_y)
pred_y = clf.predict(test_x)
#tpred_y = clf.predict(train_x)
(pred_y == test_y).sum()/len(pred_y)

In [ ]:
pickle.dump((x_arr, y_arr), open("eddsa2.pkl", "wb"))
pickle.dump((x_arr[:4000], y_arr[:4000]), open("eddsa2_train.pkl", "wb"))
pickle.dump((x_arr[4000:6000], y_arr[4000:6000]), open("eddsa2_valid.pkl", "wb"))
pickle.dump((x_arr[6000:7000], y_arr[6000:7000]), open("eddsa2_test.pkl", "wb"))

In [ ]:
onefiles = os.listdir(datadir + 'rsa-1-parsed')
onelist = []
for fname in onefiles:
    with open(datadir + 'rsa-1-parsed/' + fname, 'r') as f:
        onelist.append([int(n) for n in f])
print(len(onelist))
onethreshold = np.percentile(np.array([len(l) for l in onelist]),0.3)

zerofiles = os.listdir(datadir + 'rsa-0-parsed')
zerolist = []
for fname in zerofiles:
    with open(datadir + 'rsa-0-parsed/' + fname, 'r') as f:
        zerolist.append([int(n) for n in f])
print(len(zerolist))
zerothreshold = np.percentile(np.array([len(l) for l in zerolist]),0.3)
print((onethreshold, zerothreshold))

In [ ]:
threshold = int(min(zerothreshold, onethreshold))
newzero = [z for z in zerolist if len(z) >= threshold]
newone = [o for o in onelist if len(o) >= threshold]
dsize = min((len(newzero),len(newone)))
x_arr = np.zeros([dsize*2, threshold],dtype=np.float32)
y_arr = np.arange(dsize*2)%2
for i in range(dsize):
    x_arr[2*i] = newzero[i][:threshold]
    x_arr[2*i+1] = newone[i][:threshold]
print(x_arr.shape)

std = 16
mean = np.median(x_arr)
x_arr2 = (x_arr-mean)/std
print((mean,))

In [ ]:
train_x = x_arr2[:4000]
train_y = y_arr[:4000]
valid_x = x_arr2[4000:6000]
valid_y = y_arr[4000:6000]
test_x = x_arr2[6000:7000]
test_y = y_arr[6000:7000]
from sklearn import svm
clf = svm.SVC(kernel = 'rbf',gamma=0.001, C=10.0)

clf.fit(train_x, train_y)
pred_y = clf.predict(test_x)
#tpred_y = clf.predict(train_x)
(pred_y == test_y).sum()/len(pred_y)

In [ ]:
pickle.dump((x_arr, y_arr), open("rsa2.pkl", "wb"))
pickle.dump((x_arr[:4000], y_arr[:4000]), open("rsa2_train.pkl", "wb"))
pickle.dump((x_arr[4000:6000], y_arr[4000:6000]), open("rsa2_valid.pkl", "wb"))
pickle.dump((x_arr[6000:7000], y_arr[6000:7000]), open("rsa2_test.pkl", "wb"))